In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.callbacks import TensorBoard
from sklearn.utils import class_weight
import tensorflow_addons as tfa
import pandas as pd
import numpy as np
from multiprocessing import  Pool
from tqdm.notebook import tqdm
import nltk
from keras.layers import Dense, Dropout
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from time import time
from sklearn.metrics import classification_report
import tensorflow_addons as tfa

In [2]:
print("open dataset")
tqdm.pandas()
train = pd.read_csv("C:/Users/alhus/PycharmProjects/goodBook/dataset/goodreads_train.csv")

open dataset


In [3]:
test = pd.read_csv("C:/Users/alhus/PycharmProjects/goodBook/dataset/goodreads_test.csv")

In [4]:
train_prepro = pd.DataFrame(data=np.load(file = "C:/Users/alhus/PycharmProjects/goodBook/prepro_train_archive_PN_less.npy", allow_pickle=True), columns=['review_text'])['review_text']

In [5]:
train_prepro

0         this special book . it started slow first thir...
1         recommended . free : http : //www.audible.com/...
2         a fun , fast paced science fiction thriller . ...
3         recommended reading understand going middle am...
4         i really enjoyed book , lot recommend . it dra...
                                ...                        
899995    3.5 star . popular author agent want character...
899996    this quick read . i read lot book recently one...
899997    * spoiler alert * 3.5 star . this book sweet i...
899998    * spoiler alert another fun read ! 's new assi...
899999    * spoiler alert * 3.5 star i liked ! the story...
Name: review_text, Length: 900000, dtype: object

In [6]:
train['review_text'] = train_prepro

In [7]:
train = train[train["review_text"] != ""]

In [8]:
print("0  ",train[train['review_text'] == ""]["rating"].count())


0   0


In [9]:
rating = keras.utils.to_categorical(train['rating'], num_classes=6)
rating = rating.astype(int)

In [10]:
rating

array([[0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       ...,
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0]])

In [11]:
mylen = len(np.load('C:/Users/alhus/PycharmProjects/goodBook/voc_lemm_without_NP.npy'))

In [12]:
model = keras.Sequential()
"""inputs2 = keras.Input(shape=(1), dtype=tf.int64)
inputs3 = keras.Input(shape=(1), dtype=tf.int64)"""

model.add(keras.layers.TextVectorization(
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    output_mode='int',
    output_sequence_length=352,
    vocabulary=np.load('C:/Users/alhus/PycharmProjects/goodBook/voc_lemm_without_NP.npy')
))

In [13]:
#conc = keras.layers.concatenate([vectorize_layer])
model.add(tf.keras.layers.Embedding(input_dim=mylen+1,output_dim=32,mask_zero=True))
model.add(keras.layers.LSTM(64, input_shape= (352, 32), return_sequences=True))
model.add(keras.layers.LSTM(64, return_sequences=True))
model.add(keras.layers.LSTM(64))
model.add(Dropout(0.2))
model.add(keras.layers.Dense(128, activation=tf.keras.activations.relu))


model.add(keras.layers.Dense(6, activation=tf.keras.activations.softmax))

In [14]:

tensorboard = TensorBoard(log_dir="C:/Users/alhus/PycharmProjects/goodBook/logs/lstm3".format(time()))

In [15]:
#for learning_rate in learning_rates:
model.compile(optimizer=tf.keras.optimizers.Adamax(learning_rate=0.01),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=[tf.keras.metrics.categorical_accuracy, tfa.metrics.F1Score(num_classes=6, average="weighted")]
              )

In [16]:
#class_weights = class_weight.compute_class_weight(class_weight='balanced',classes= np.unique(train['rating']), y = train['rating'])
#weight = {i : class_weights[i] for i in range(6)}

In [17]:
model.fit(train['review_text'], rating, epochs=10,
                  callbacks=[
                      tf.keras.callbacks.TensorBoard(log_dir="C:/Users/alhus/PycharmProjects/goodBook/logs/lstm3"),
                  ],
                  batch_size=500, shuffle= True, validation_split=0.2
                  )
model.summary()

C:\Users\alhus\anaconda3\envs\Book_Review_Project\lib\site-packages\keras\engine\data_adapter.py:1699: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


Epoch 1/10
1440/1440 [==============================] - 1579s 1s/step - loss: 1.1341 - categorical_accuracy: 0.5121 - f1_score: 0.5028 - val_loss: 1.0429 - val_categorical_accuracy: 0.5515 - val_f1_score: 0.5464
Epoch 2/10
1440/1440 [==============================] - 1586s 1s/step - loss: 1.0227 - categorical_accuracy: 0.5620 - f1_score: 0.5576 - val_loss: 1.0008 - val_categorical_accuracy: 0.5693 - val_f1_score: 0.5653
Epoch 3/10
1440/1440 [==============================] - 1584s 1s/step - loss: 0.9697 - categorical_accuracy: 0.5876 - f1_score: 0.5844 - val_loss: 0.9772 - val_categorical_accuracy: 0.5825 - val_f1_score: 0.5778
Epoch 4/10
1440/1440 [==============================] - 1590s 1s/step - loss: 0.9372 - categorical_accuracy: 0.6026 - f1_score: 0.5998 - val_loss: 0.9745 - val_categorical_accuracy: 0.5855 - val_f1_score: 0.5794
Epoch 5/10
1440/1440 [==============================] - 2121s 1s/step - loss: 0.9079 - categorical_accuracy: 0.6154 - f1_score: 0.6130 - val_loss: 0.970

In [18]:
test_prepro = pd.DataFrame(data=np.load(file="C:/Users/alhus/PycharmProjects/goodBook/prepro_test_archive_PN_less.npy", allow_pickle=True), columns=['review_text'])['review_text']
test['review_text'] = test_prepro

In [19]:
restest = model.predict([test['review_text']])

 7125/14939 [=============>................] - ETA: 4:02


KeyboardInterrupt



In [ ]:
ff = []
for line in tqdm(restest):
    tmp = -2
    category = None
    for i in (range(6)):
        if line[i] > tmp:
            category = i
            tmp = line[i]
    ff.append(category)
data = np.array(ff)

In [ ]:
test['rating'] = data

In [ ]:
id = test['review_id'].to_numpy()
rating = test['rating'].to_numpy()
df = pd.DataFrame( columns=['review_id', 'rating'])

In [ ]:
df['review_id'] = id
df['rating'] = rating

In [ ]:
df.to_csv('submission_lstm12_model.csv',index=False )

In [ ]:
model.summary()

In [ ]:
from keras import backend as K

In [ ]:
K.clear_session

In [ ]:
del model

In [ ]:
print("0  ",train[train['review_text'] == ""]["rating"].count())
